In [4]:
import sys
sys.path.append('../../utils/')
from coordinatesConversions import *
import numpy as np
from scipy.constants import m_p, c
import pickle

The action for the y-plane (the same applies for the x-plane)
\begin{equation}
J_y = \frac{1}{2}(y_n^2 +yp_n^2) \ \ (1)
\end{equation}

where 

\begin{equation}
y_n = \frac{y}{\sqrt{\beta_y}} \ \ (2)
\end{equation}

and


\begin{equation}
yp_n = \frac{\alpha_y y}{\sqrt{\beta_y}} + \sqrt{\beta_y} yp \ \ (3)
\end{equation}

The detuning with amplitude is computed by:

\begin{equation}
\Delta Q_x = \alpha_{xx} J_x + \alpha_{xy} J_y  \ \ (4)
\end{equation}

\begin{equation}
\Delta Q_y = \alpha_{yy}  J_y + \alpha_{yx} J_x  \ \ (5)
\end{equation}

where $\alpha_{xx}, \alpha_{yy}$ and $\alpha_{xy}=\alpha_{xy}$ (1/m) are the detuning coefficients. 



For the **rms tune spread** and assuming that the coupling coefficient $\alpha_{xy}$ is zero it applies:

\begin{equation}
Var(\Delta Q_y) = \langle \Delta Q_y ^2 \rangle - \langle \Delta Q_y \rangle^2 =  \langle \alpha_{yy}^2 J_y ^2 \rangle - \langle  \alpha_{yy} J_y \rangle^2 = \alpha_{yy}^2 ( \langle \Delta J_y ^2 \rangle - \langle \Delta J_y \rangle ^2 ) =  \alpha_{yy}^2 Var(J_y) \ \ (6)
\end{equation}

where <> corresponds to np.mean().

Thus from (6) we write:

\begin{equation}
rms(\Delta Q_y) = \sqrt{Var(\Delta Q_y)} = \sqrt{ \alpha_{yy}^2 Var(J_y)} = \alpha_{yy} rms(J_y) \ \ (7)
\end{equation}

where rms corresponds to np.std()


In [5]:
def amplitude_detuning_x(Jx, Jy,a_xx, a_xy):
        return a_xx*2*Jx+a_xy*2*Jy
    
def amplitude_detuning_y(Jx, Jy, a_yy, a_xy):
    return a_yy*2*Jy+a_xy*2*Jx

### Load the initial distribution used for the sixtracklib simulations and the opitc functions at the location of the CC2


In [6]:
initial_distribution = pickle.load(open('./input/initial_coordinates.pkl', 'rb'))
twiss = pickle.load(open('input/twiss_sanity_check.pkl', 'rb'))

In [7]:
# Optics at CC2
beta_y = twiss['betay_cc2']
beta_x = twiss['betax_cc2']
alpha_y = twiss['alphay_cc2']
alpha_x = twiss['alphax_cc2']
  
# normalised emittance
e_norm_x, e_norm_y = 2e-6, 2e-6 # [m]

# Coordinates
x, px = initial_distribution['x'], initial_distribution['px']
y, py = initial_distribution['y'], initial_distribution['py']

# Normalised coordinates 
x_n, px_n = cmpt_normalised_coordinates(x, px, beta_x, alpha_x)
y_n, py_n = cmpt_normalised_coordinates(y, py, beta_y, alpha_y)


# Compute actions
Jx_init = cmpt_actions(x_n, px_n)
Jy_init = cmpt_actions(y_n, py_n)

print(f'Initial <Jx> = {np.mean(Jx_init)} m')
print(f'Initial <Jy> = {np.mean(Jy_init)} m')

Initial <Jx> = 7.469898989251154e-09 m
Initial <Jy> = 6.945867993376703e-09 m


### Sanity check:  <J> should be equal with the geometric emittance

In [8]:
# Compute gemoteric emittance 
p0c = 269.99e9 # [eV]
E_rest = 938272081.0 
E_0 = np.sqrt(p0c**2+E_rest**2)
gamma_0 =  E_0/E_rest # gamma realtivistic of the reference particle  
beta_0 = np.sqrt(1-1/gamma_0**2) # beta realtivistic of the reference particle
e_geom_y = e_norm_y/(gamma_0*beta_0)
e_geom_x = e_norm_x/(gamma_0*beta_0)

print(f'e_geom_x= {e_geom_x} m, e_geom_y= {e_geom_y} m')
print('The e_geom_x is not in exact agreement with <Jx>. Could this be because the x-dispersion is not taken into account? ')

e_geom_x= 6.950420985962443e-09 m, e_geom_y= 6.950420985962443e-09 m
The e_geom_x is not in exact agreement with <Jx>. Could this be because the x-dispersion is not taken into account? 


## Compute RMS tune spread accoridng to Eq.7

### Α) sanity check that np.std(J) = np.sqrt(var(j) = np.sqrt(<J^2> - <J>^2>)

In [14]:
print(f'np.std(Jy) = {np.std(Jy_init)}')
print(f'np.sqrt(Var((Jy)) = {np.sqrt(np.mean(Jy_init**2)- np.mean(Jy_init)**2)}')

np.std(Jy) = 6.969922129199802e-09
np.sqrt(Var((Jy)) = 6.969922129199802e-09


In [55]:
# Detuning coefficients in 1/m
axx = 0.0
ayy = -49.16694908 #
axy =  -402.9517487 #0.0

In [56]:
rms_Jx_init = np.std(Jx_init)
rms_Jy_init = np.std(Jy_init)

In [57]:
Dqy_rms = amplitude_detuning_y(rms_Jx_init, rms_Jy_init, ayy, axy)
print(f'rms(DeltaQy) = {Dqy_rms}')

rms(DeltaQy) = -6.640528290348846e-06


In [58]:
1/(Dqy_rms*43.45e3)

-3.465832644259609

## Some other tests

### 1) Consider:

\begin{equation}
rms(J_y) =  \frac{1}{2}(rms(y_n)^2 +rms(yp_n)^2) 
\end{equation}

In [42]:
print(f'rms yn = {np.std(y_n)}')
print(f'rms ypn {np.std(py_n)}')
print('rms Jy _2 = {}'.format((1/2)*(np.std(y_n)**2+ np.std(py_n)**2)))

rms yn = 8.458632657192202e-05
rms ypn 8.207485721393128e-05
rms Jy _2 = 6.945564414809524e-09


## Very close to the one calculated before from the distribution 